# Toxicity classification using BERT

**Description:** This notebook builds a model to classify a text comment as toxic/non-toxic using BERT.
The data used for training the model was originally sourced from [Kaggle Toxic Commnet Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge). 

<a id = 'returnToTop'></a>

## Notebook Contents
  * 1. [Setup](#setup) 
  * 2. [Data](#data)  
  * 3. [Tokenization](#tokenization)
  * 4. [Model Training](#training)
  * 5. [Model Evaluation](#evaluation)

[Return to Top](#returnToTop)  
<a id = 'setup'></a>

## 1. Setup

Install tensorflow, pydot and transformers

In [1]:
!pip install tensorflow-datasets --quiet

In [2]:
!pip install pydot --quiet

In [3]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.2 MB/s eta 0:00:00


Import required libraries

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

In [5]:
from transformers import BertTokenizer, TFBertModel

Helper function to plot histories.

In [6]:
# 4-window plot of loss and accuracy for two models, for comparison

def make_plot(axs,
              model_history1, 
              model_history2, 
              model_1_name='model 1',
              model_2_name='model 2',
              ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    for i, metric in enumerate(['loss', 'accuracy']):
        y_lim_lower1 = np.min(model_history1.history[metric])
        y_lim_lower2 = np.min(model_history2.history[metric])
        y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

        y_lim_upper1 = np.max(model_history1.history[metric])
        y_lim_upper2 = np.max(model_history2.history[metric])
        y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

        for j, model_history in enumerate([model_history1, model_history2]):
            model_name = [model_1_name, model_2_name][j]
            ax1 = axs[i, j]
            ax1.plot(model_history.history[metric])
            ax1.plot(model_history.history['val_%s' % metric])
            ax1.set_title('%s - %s' % (metric, model_name))
            ax1.set_ylabel(metric, bbox=box)
            ax1.set_ylim(y_lim_lower, y_lim_upper)

Function to calculate cosine similarity

In [7]:
def cosine_similarities(vecs):
    for v_1 in vecs:
        similarities = ''
        for v_2 in vecs:
            similarities += ('\t' + str(np.dot(v_1, v_2)/np.sqrt(np.dot(v_1, v_1) * np.dot(v_2, v_2)))[:4])
        print(similarities)

[Return to Top](#returnToTop)  
<a id = 'data'></a>

## 2. Data

The jigsaw database has been downloaded from kaggle, cleaned and preprocessed and split into train, validation and test datasets. The datsets are stored on amazon S3 where we will be accessing them from.

In [8]:
df_train = pd.read_csv("https://adamhyman-public.s3.amazonaws.com/train_data.csv")

df_valid = pd.read_csv("https://adamhyman-public.s3.amazonaws.com/validation_data.csv")

df_test = pd.read_csv("https://adamhyman-public.s3.amazonaws.com/test_data.csv")

In [9]:
df_train.head()

,comment_text,toxic
0,Sobriety would also be a good first step to en...,0
1,gender is NOT a feeling or belief you ignorant...,1
2,"""If Whitmer had been a white woman at a weddin...",1
3,Nobody stated it was a male thing. Take a brea...,0
4,Ontario deserves this for the government they ...,1


In [40]:
#There are 3 null comments in train dataset, 1 in valid and 1 in test. They need to be removed or we get error while convertin gto tensor
df_train = df_train.dropna(how='any',axis=0) 
df_valid = df_valid.dropna(how='any',axis=0) 
df_test = df_test.dropna(how='any',axis=0) 

In [41]:
df_train.head()

,comment_text,toxic
0,Sobriety would also be a good first step to en...,0
1,gender is NOT a feeling or belief you ignorant...,1
2,"""If Whitmer had been a white woman at a weddin...",1
3,Nobody stated it was a male thing. Take a brea...,0
4,Ontario deserves this for the government they ...,1


In [42]:
train_comments, train_labels = df_train["comment_text"], df_train["toxic"]
valid_comments, valid_labels = df_valid["comment_text"], df_train["toxic"]
test_comments, test_labels = df_test["comment_text"], df_train["toxic"]

In [43]:
train_comments, train_labels = tf.convert_to_tensor(train_comments), tf.convert_to_tensor(train_labels)
valid_comments, valid_labels = tf.convert_to_tensor(valid_comments), tf.convert_to_tensor(valid_labels)
test_comments, test_labels = tf.convert_to_tensor(test_comments), tf.convert_to_tensor(test_labels)

In [44]:
train_comments[:4]

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'Sobriety would also be a good first step to ending rape on reserves.',
       b'gender is NOT a feeling or belief you ignorant alt left lunatic,,',
       b'"If Whitmer had been a white woman at a wedding reception with other white guests and had given her statement, I don\xe2\x80\x99t think we would ever have heard of the Fairbanks Four"................I\'m not being thin skinned. Here\'s a quote where Charles Wohlforth is clearly blaming whites for unfair treatment of natives. Read the story, carefully. It is all about blaming whites for unfair treatment of natives. The Fairbanks Four are just the actors in Charles\'s real story and I will title it for him "Whites Treat Natives Unfairly ."..............Nonsense!',
       b"Nobody stated it was a male thing. Take a breath and quit trying to win a debate and ask yourself - when was the last time a female you know cared enough about the sport to discuss a receivers' route running? It

In [45]:
train_labels[:4]

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([0, 1, 1, 0])>

[Return to Top](#returnToTop)  
<a id = 'tokenization'></a>
## 3. Tokenization

Get the pre-trained BERT model and tokenizer.

In [46]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [53]:
# BERT Tokenization of training and validation data
num_train_examples = -1      # set number of train examples - 1500 for realtime demo
num_valid_examples = -1        # set number of test examples - 500 for realtime demo

MAX_SEQUENCE_LENGTH = 128                 # set max_length of the input sequence

train_examples = [x.decode('utf-8') for x in train_comments.numpy()]
valid_examples = [x.decode('utf-8') for x in valid_comments.numpy()]

x_train = bert_tokenizer(train_examples[:num_train_examples],
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_train = train_labels[:num_train_examples]

x_valid = bert_tokenizer(valid_examples[:num_valid_examples],
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_valid = valid_labels[:num_valid_examples]

[Return to Top](#returnToTop)  
<a id = 'model'></a>

# 4. Model definition and training


Now we define the model...

In [54]:
def create_bert_classification_model(bert_model,
                                     num_train_layers=0,
                                     hidden_size = 200, 
                                     dropout=0.3,
                                     learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes
    """
    if num_train_layers == 0:
        # Freeze all layers of pre-trained BERT model
        bert_model.trainable = False

    elif num_train_layers == 12: 
        # Train all layers of the BERT model
        bert_model.trainable = True

    else:
        # Restrict training to the num_train_layers outer transformer layers
        retrain_layers = []

        for retrain_layer_number in range(num_train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)
          
        
        print('retrain layers: ', retrain_layers)

        for w in bert_model.weights:
            if not any([x in w.name for x in retrain_layers]):
                #print('freezing: ', w)
                w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}      

    bert_out = bert_model(bert_inputs)

    pooler_token = bert_out[1]
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')
    
    return classification_model

In [55]:
bert_classification_model = create_bert_classification_model(bert_model, num_train_layers=0)

In [56]:
#confirm all layers are frozen
bert_classification_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 128)]       0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 128)]       0           []                               
 er)                                                                                              
                                                                                            

In [57]:
bert_classification_model_history = bert_classification_model.fit(
    [x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
    y_train,
    validation_data=([x_valid.input_ids, x_valid.token_type_ids, x_valid.attention_mask], y_valid),
    batch_size=32,
    epochs=1
)  

5413/5413 [==============================] - ETA: 0s - loss: 0.6385 - accuracy: 0.6328

ValueError: ignored

[Return to Top](#returnToTop)  
<a id = 'evaluation'></a>

# 4. Model Evaluation

In [ ]:
#verify some predictions
test = bert_tokenizer(["Racism is bad"],
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

bert_classification_model.predict(x=test)

In [ ]:
#Prepare test data
test_examples = [x.decode('utf-8') for x in test_comments.numpy()]

x_test = bert_tokenizer(test_examples,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')
y_test = test_labels

In [ ]:
# run the tr)ained model on the test data (the model outputs probabilities)
y_test_predictions = bert_classification_model.predict(x=x_test).flatten()

# apply the threshold function to create a 0, 1 outcome
y_test_pred = np.where(y_test_predictions>=0.5, 1, 0)
y_test_pred[:10] # first 10 only

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_test_pred.numpy())